# Topic modelling and entity extraction

In [73]:
# Usual imports
import numpy as np
import pandas as pd
import string
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
# !python -m spacy download en_core_web_lg

ModuleNotFoundError: No module named 'gensim'

In [53]:
CLEAN_AND_TOKENIZE = True

INPUT_PATH = os.path.join(os.pardir, 'Datasets')
OUT_PATH = os.path.join(os.pardir, 'Datasets')
INPUT_FILE = 'train.csv'
SAMPLE_SIZE = 50000

NUM_TOPICS = 10

In [54]:
# Loading data
df = pd.read_csv(os.path.join(INPUT_PATH, INPUT_FILE), nrows=SAMPLE_SIZE)
df.set_index('id', inplace=True)
df.fillna('Empty question', inplace=True)
df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [57]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [58]:
if CLEAN_AND_TOKENIZE:
    df['question1'] = df['question1'].apply(spacy_tokenizer)
    df['question2'] = df['question2'].apply(spacy_tokenizer)

## Entity Extraction

In [59]:
# Creating a spaCy object
nlp = spacy.load('en_core_web_lg')

Single example:

In [62]:
ex = nlp(df['question1'][98])
spacy.displacy.render(ex, style='ent',jupyter=True)

Full dataset:

In [63]:
def get_entities(x):
    entity_types = (
        'PERSON',
        'NORP',
        'FAC',
        'ORG',
        'GPE',
        'LOC',
        'PRODUCT',
        'EVENT',
        'WORK_OF_ART',
        'LAW',
        'LANGUAGE',
        'DATE',
        'TIME',
        'PERCENT',
        'MONEY',
        'QUANTITY',
        'ORDINAL',
        'CARDINAL'
    )
    
    df = pd.DataFrame(
        data=np.zeros([len(x), len(entity_types)]).astype(int),
        columns=entity_types,
        index=x.index
    )
    for i, val in x.iteritems():
        if i % (len(x) / 10) == 0:
            print('Entry {}/{}'.format(i, len(x)))
        doc = nlp(spacy_tokenizer(val), disable=['parser'])
        for ent in doc.ents:
            df.loc[i, ent.label_] += 1
    df.columns = [col + '_COUNT' for col in df.columns]
    return df

In [64]:
ents_df_1 = get_entities(df['question1'])
ents_df_2 = get_entities(df['question2'])
ents_diff = ents_df_1 - ents_df_2
assert ents_diff.index.nunique() == len(ents_diff), 'Index not unique'
ents_diff.to_csv(os.path.join(OUT_PATH, 'FEATURE_entity_counts.csv'))

Entry 0/50000
Entry 5000/50000
Entry 10000/50000
Entry 15000/50000
Entry 20000/50000
Entry 25000/50000
Entry 30000/50000
Entry 35000/50000
Entry 40000/50000
Entry 45000/50000
Entry 0/50000
Entry 5000/50000
Entry 10000/50000
Entry 15000/50000
Entry 20000/50000
Entry 25000/50000
Entry 30000/50000
Entry 35000/50000
Entry 40000/50000
Entry 45000/50000


## Parts of Speech tagging

In [65]:
# # POS tagging
# for i in nlp(review):
#     print(i,"=>",i.pos_)

## Topic-modelling

In [66]:
# Creating a vectorizer
vectorizer = CountVectorizer(
    min_df=5, 
    max_df=0.9, 
    stop_words='english', 
    lowercase=True, 
    token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}'
)
corpus = pd.concat([df['question1'], df['question2']])
vectorized_corpus = vectorizer.fit_transform(corpus)
q1_vectorized = vectorizer.transform(df['question1'])
q2_vectorized = vectorizer.transform(df['question2'])

In [67]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(
    n_components=NUM_TOPICS, 
    max_iter=10, 
    verbose=True,
    random_state=0
)
lda.fit(vectorized_corpus)
q1_lda = lda.transform(q1_vectorized)
q2_lda = lda.transform(q2_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [68]:
topic_similarity = q1_lda * q2_lda
topic_df = pd.DataFrame(
    data=topic_similarity,
    columns=['PROB_BOTH_SHARE_TOPIC_' + str(i) for i in range(topic_similarity.shape[1])],
    index=df.index
)
topic_df.to_csv(os.path.join(OUT_PATH, 'FEATURE_topic_sharing.csv'))

In [69]:
topic_df.head()

,PROB_BOTH_SHARE_TOPIC_0,PROB_BOTH_SHARE_TOPIC_1,PROB_BOTH_SHARE_TOPIC_2,PROB_BOTH_SHARE_TOPIC_3,PROB_BOTH_SHARE_TOPIC_4,PROB_BOTH_SHARE_TOPIC_5,PROB_BOTH_SHARE_TOPIC_6,PROB_BOTH_SHARE_TOPIC_7,PROB_BOTH_SHARE_TOPIC_8,PROB_BOTH_SHARE_TOPIC_9
id,,,,,,,,,,
0,0.000179,0.000179,0.000179,0.000179,0.000179,0.001964,0.664464,0.000179,0.000179,0.000179
1,0.000556,0.000556,0.000556,0.000556,0.011667,0.000556,0.128333,0.006111,0.000556,0.006111
2,0.000286,0.000286,0.034571,0.000286,0.000286,0.305993,0.000286,0.000286,0.000286,0.003143
3,0.000500,0.020500,0.000500,0.000500,0.000500,0.000500,0.015500,0.000500,0.000500,0.000500
4,0.000250,0.030249,0.000250,0.395254,0.000250,0.000250,0.000250,0.002750,0.000250,0.000250


In [70]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [71]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('good', 3409.307142278148), ('way', 3217.075125788998), ('learn', 1720.3605737193564), ('start', 1543.3585665382416), ('want', 1416.0468971222376), ('feel', 1270.5732357857507), ('buy', 1001.1150715571399), ('girl', 928.3349284319263), ('person', 859.8524973607471), ('stop', 849.3465175210454)]
Topic 1:
[('like', 3314.766080513468), ('job', 1392.9958490821577), ('live', 897.6805499693997), ('help', 721.3931393154149), ('place', 693.8916388329501), ('math', 642.4429974876906), ('video', 613.3276711712149), ('google', 608.6539645148972), ('software', 565.3066171682856), ('bad', 546.3823265472535)]
Topic 2:
[('time', 2099.822364160718), ('year', 1864.8161738274844), ('difference', 1837.2759309418432), ('doe', 1712.6513252080438), ('mean', 1553.7560450325666), ('book', 1444.164188681905), ('love', 1132.180040813065), ('old', 919.8394864817185), ('improve', 863.266936301231), ('woman', 859.5473070510342)]
Topic 3:
[('money', 1773.1930470195214), ('movie', 1305.42528512

The index in the above list with the largest value represents the most dominant topic for the given review.


# Visualizing LDA results with pyLDAvis

In [72]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, q1_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2     -367.960510 -213.068008       1        1  10.481291
5      180.153427 -283.704468       2        1  10.453228
0      135.331894  244.676605       3        1  10.407971
4       96.131157  -36.401863       4        1  10.211719
1     -109.438972   97.462044       5        1   9.879227
9     -369.286804   79.572792       6        1   9.800612
7      -89.384140 -399.099976       7        1   9.787193
6     -122.715385 -147.683182       8        1   9.751104
3     -161.939529  341.060791       9        1   9.655237
8      341.203308    7.842676      10        1   9.572419, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
2093  Default  2583.000000         doe  2583.000000  30.0000  30.0000
3377  Default  1817.000000       india  1817.000000  29.0000  29.0000
3970  Default  1602.000000        like  1602.000000  28.0000  28.0000
7358  Default  1606.000000         use  1606.000000  27.0000  27.0000
2902  Default  1594.000000        good  1594.000000  26.0000  26.0000
7580  Default  1504.000000         way  1504.000000  25.0000  25.0000
5072  Default  1645.000000      people  1645.000000  24.0000  24.0000
5591  Default  1081.000000       quora  1081.000000  23.0000  23.0000
3962  Default  1047.000000        life  1047.000000  22.0000  22.0000
3800  Default  1009.000000        know  1009.000000  21.0000  21.0000
7685  Default   996.000000        work   996.000000  20.0000  20.0000
7039  Default  1011.000000        time  1011.000000  19.0000  19.0000
7730  Default   898.000000        year   898.000000  18.0000  18.0000
4457  Default   857.000000       money   857.000000  17.0000  17.0000
1968  Default   885.000000  difference   885.000000  16.0000  16.0000
7000  Default   839.000000       think   839.000000  15.0000  15.0000
3898  Default  1091.000000       learn  1091.000000  14.0000  14.0000
6998  Default   785.000000       thing   785.000000  13.0000  13.0000
3378  Default   770.000000      indian   770.000000  12.0000  12.0000
5582  Default   752.000000    question   752.000000  11.0000  11.0000
4260  Default   748.000000        mean   748.000000  10.0000  10.0000
6613  Default   721.000000       start   721.000000   9.0000   9.0000
831   Default   696.000000        book   696.000000   8.0000   8.0000
3679  Default   673.000000         job   673.000000   7.0000   7.0000
7193  Default   644.000000       trump   644.000000   6.0000   6.0000
7549  Default   662.000000        want   662.000000   5.0000   5.0000
3029  Default   651.000000      happen   651.000000   4.0000   4.0000
4508  Default   631.000000       movie   631.000000   3.0000   3.0000
1779  Default   637.000000         day   637.000000   2.0000   2.0000
4831  Default   581.000000      online   581.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3852  Topic10   470.428712    language   470.865813   2.3454  -3.7952
4041  Topic10   443.628604        long   444.065733   2.3453  -3.8538
6691  Topic10   424.822169     student   425.259281   2.3453  -3.8971
2104  Topic10   395.357085      donald   395.794178   2.3452  -3.9690
1349  Topic10   380.337864     college   380.774978   2.3451  -4.0077
3110  Topic10   351.720399        high   352.157528   2.3450  -4.0860
5449  Topic10   339.244291   programme   339.681387   2.3450  -4.1221
5682  Topic10   320.739416        read   321.176536   2.3449  -4.1782
7711  Topic10   257.662738       wrong   258.099860   2.3446  -4.3971
612   Topic10   254.145606        bank   254.582736   2.3446  -4.4109
3035  Topic10   225.521995        hard   225.959119   2.3443  -4.5304
4008  Topic10   225.340851      little   225.777989   2.3443  -4.5312
7191  Topic10   221.217944        true   221.655071   2.3443  -4.5496
6021  Topic10   195.951886         run   

## How to interpret this graph?
1. Topics on the left while their respective keywords are on the right.
2. Larger topics are more frequent and closer the topics, mor the similarity
3. Selection of keywords is based on their frequency and discriminancy.

**Hover over the topics on the left to get information about their keywords on the right.**